In [1]:
import cupy as cp
from scipy.optimize import linear_sum_assignment
from datetime import time
from numpy.dual import inv

C:\Users\data\AppData\Local\Temp\ipykernel_15592\2758445751.py:4: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import inv


# 1. 中心点の計算

In [2]:
def center_pt_batched(bboxes):
    # Ensure bboxes is a cupy array
    bboxes = cp.array(bboxes) if not isinstance(bboxes, cp.ndarray) else bboxes
    
    # Return an empty array for no bboxes
    if bboxes.size == 0:
        return cp.array([])
    
    # Ensure all points are non-negative
    bboxes = cp.clip(bboxes, 0, None)

    # If the bbox is a single box, convert it to a 2D array
    if len(bboxes.shape) == 1:
        bboxes = cp.expand_dims(bboxes, axis=0)
    
    # Calculate center points
    centers_x = (bboxes[:, 0] + bboxes[:, 2]) / 2
    centers_y = (bboxes[:, 1] + bboxes[:, 3]) / 2
    
    # Combine x and y centers into a single array
    centers = cp.stack((centers_x, centers_y), axis=-1)
    
    return centers

Unit test

In [3]:
# Corrected test cases for center_pt_batched function
assert cp.array_equal(center_pt_batched(cp.array([[0, 0, 10, 10]])), cp.array([[5, 5]])), "Test 1 failed"
assert cp.array_equal(center_pt_batched(cp.array([[-5, -5, 5, 5]])), cp.array([[2.5, 2.5]])), "Test 2 failed"
assert cp.array_equal(center_pt_batched(cp.array([[0, 0, 0, 0]])), cp.array([[0, 0]])), "Test 3 failed"
assert cp.array_equal(center_pt_batched(cp.array([[10, 10, 20, 20]])), cp.array([[15, 15]])), "Test 4 failed"

assert cp.array_equal(center_pt_batched(cp.array([[0, 0, 10, 10], [-5, -5, 5, 5]])), cp.array([[5, 5], [2.5, 2.5]])), "Test 5 failed"
assert cp.array_equal(center_pt_batched([0, 0, 10, 10]), cp.array([[5, 5]])), "Test 6 failed"

print("All tests passed")

All tests passed


# 2.二点間の距離の測定

In [4]:
def distance_pts_batched_matrix(pts1, pts2):
    # Ensure pts1 and pts2 are cupy arrays
    pts1 = cp.array(pts1) if not isinstance(pts1, cp.ndarray) else pts1
    pts2 = cp.array(pts2) if not isinstance(pts2, cp.ndarray) else pts2
    
    # Return an empty array for no points
    if pts1.size == 0 or pts2.size == 0:
        return cp.array([])
    
    # Ensure all points are non-negative
    pts1 = cp.clip(pts1, 0, None)
    pts2 = cp.clip(pts2, 0, None)
    
    # Ensure pts1 and pts2 are 2D arrays
    if len(pts1.shape) == 1:
        pts1 = cp.expand_dims(pts1, axis=0)
    if len(pts2.shape) == 1:
        pts2 = cp.expand_dims(pts2, axis=0)

    # If the size of pts1 and pts2 are not the same, pad the smaller array with zeros
    if pts1.shape[0] < pts2.shape[0]:
        pts1 = cp.vstack((pts1, cp.zeros((pts2.shape[0] - pts1.shape[0], pts1.shape[1]))))
    elif pts1.shape[0] > pts2.shape[0]:
        pts2 = cp.vstack((pts2, cp.zeros((pts1.shape[0] - pts2.shape[0], pts2.shape[1]))))
    
    # Calculate distance matrix
    diff_x = pts1[:, cp.newaxis, 0] - pts2[cp.newaxis, :, 0]
    diff_y = pts1[:, cp.newaxis, 1] - pts2[cp.newaxis, :, 1]
    dist_matrix = diff_x**2 + diff_y**2
    
    return dist_matrix

Unit test

In [5]:
pts1 = cp.array([[-1, -1], [1, 1], [2, 2]])       # horizontal axis
pts2 = cp.array([[0, 0], [1, 1], [2, 2], [3, 3]]) # vertical axis

# Corrected test cases for distance_pts_batched_matrix function
matrix = distance_pts_batched_matrix(pts1, pts2)

# Expected output
#         (0, 0) (1, 1) (2, 2) (3, 3) <- pts2
# (-1, -1) 2     1      8      18  <-- horizontal array, elements are clipped to zero because of negative values
# (1, 1)   2     0      2      8
# (2, 2)   8     2      0      2
# (0, 0)   0     2      8      18  <-- horizontal array, elements are padded with zeros because of size mismatch
# pts1

print(matrix)

[[ 0.  2.  8. 18.]
 [ 2.  0.  2.  8.]
 [ 8.  2.  0.  2.]
 [ 0.  2.  8. 18.]]


# 2. 追跡と検出

In [6]:
def associate_detections_to_trackers_gpu(detected_bboxes, tracking_pts, dist_variance=500):

    # Ensure detected_bboxes and tracking_pts are cupy arrays
    detected_bboxes = cp.array(detected_bboxes) if not isinstance(detected_bboxes, cp.ndarray) else detected_bboxes
    tracking_pts = cp.array(tracking_pts) if not isinstance(tracking_pts, cp.ndarray) else tracking_pts

    if detected_bboxes.size == 0:
        return [], [], list(range(len(tracking_pts)))
    
    if tracking_pts.size == 0:
        return [], list(range(len(detected_bboxes))), []

    # Convert detected_bboxes to center points
    # Input shape: (num_detections, 4)
    # Output shape: (num_detections, 2)
    detected_centers = center_pt_batched(detected_bboxes)

    # Generate distance variance matrix
    # Input shape of detected_centers: (num_detections, 2)
    # Input shape of tracking_pts: (num_trackers, 2)
    # Output shape: (num_detections, num_trackers)
    dist_matrix = distance_pts_batched_matrix(detected_centers, tracking_pts)

    # Convert the CuPy array to a NumPy array before passing it to linear_sum_assignment
    row_ind, col_ind = linear_sum_assignment(dist_matrix.get())
    # row_ind is the index of the detected_centers
    # col_ind is the index of the tracking_pts

    # Debugging
    # print(row_ind, col_ind)

    matched_pairs = []
    untracked_detections = []
    untracked_trackers = []

    # Iterate through the row and column indices to find the matched pairs
    for i, j in zip(row_ind, col_ind):
        # If i or j is padded with zeros, skip the pair
        if i >= detected_centers.shape[0] or j >= tracking_pts.shape[0]:
            continue

        # Check if the distance between the detected center and the tracking point is less than the distance variance
        if dist_matrix[i, j] <= dist_variance:
            matched_pairs.append((i, j))
        else:
            untracked_detections.append(i)
            untracked_trackers.append(j)

    # Find the unmatched detections by checking if the index is in matched_pairs
    untracked_detections += [i for i in range(detected_centers.shape[0]) if i not in [pair[0] for pair in matched_pairs]]

    # Find the unmatched trackers by checking if the index is in matched_pairs
    untracked_trackers += [j for j in range(tracking_pts.shape[0]) if j not in [pair[1] for pair in matched_pairs]]
    
    return matched_pairs, untracked_detections, untracked_trackers


Unit test

In [7]:
# horizontal axis > vertical axis
detected_bboxes = [[0, 0, 10, 10], [-5, -5, 5, 5], [10, 10, 20, 20], [0, 0, 0, 0], [1, 1, 15, 15]] # horizontal axis
tracking_pts = [[1,1], [2,2], [3,3]] # vertical axis
print(associate_detections_to_trackers_gpu(detected_bboxes, tracking_pts))

# horizontal axis < vertical axis
detected_bboxes = [[0, 0, 10, 10], [1, 1, 15, 15]] # horizontal axis
tracking_pts = [[1,1], [2,2], [3,3], [4,4]] # vertical axis
print(associate_detections_to_trackers_gpu(detected_bboxes, tracking_pts))

([(0, 0), (2, 2), (4, 1)], [1, 3], [])
([(0, 2), (1, 3)], [], [0, 1])


# 3. Kalman Filter

In [8]:
class KalmanFilter:
    def __init__(self, A, B, H, Q, R, x0, P0):
        self.A = cp.asarray(A)
        self.B = cp.asarray(B)
        self.H = cp.asarray(H)
        self.Q = cp.asarray(Q)
        self.R = cp.asarray(R)
        self.x = cp.asarray(x0)
        self.P = cp.asarray(P0)

        self.last_update_time = 0

    def predict(self, u=None):
        if u is None:
            u = cp.zeros((self.B.shape[1], 1))
        self.x = self.A @ self.x + self.B @ u
        self.P = self.A @ self.P @ self.A.T + self.Q

    def update(self, z):
        z = cp.asarray(z)
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ cp.asarray(inv(cp.asnumpy(S)))
        self.x = self.x + K @ y
        self.P = self.P - K @ self.H @ self.P

        self.last_update_time = time.time()

    def get_state(self):
        return cp.asnumpy(self.x)

# 4. カルマンフィルターを利用し、バッチ軌道予測

In [ ]:
def track_objects_batch(kalman_filter_class, detections, A, B, H, Q, R, x0, P0):
    kf_list = [kalman_filter_class(A, B, H, Q, R, x0, P0) for _ in range(detections.shape[1])]
    results = []

    for t in range(detections.shape[0]):
        batch_results = []
        for i, kf in enumerate(kf_list):
            z = detections[t, i].reshape((2, 1))
            kf.predict()
            kf.update(z)
            state = kf.get_state()
            batch_results.append(state.ravel().tolist())
        results.append(batch_results)

    return cp.asarray(results)

# 4. 追跡クラス

In [ ]:
class Sort:

    """
    kalman_filter_class: Kalman filter class to use
    max_age: Maximum time to keep a track alive without an update

    """

    def __init__(self, kalman_filter_class, max_age=1, variance_threshold=500, kalman_filter_params=None):
        self.kalman_filter_class = kalman_filter_class
        self.max_age = max_age
        self.variance_threshold = variance_threshold
        self.trackers = []

        if kalman_filter_params is not None:
            self.kalman_filter_params = kalman_filter_params
        else:
            self.kalman_filter_params = {
                'A': cp.eye(4),
                'B': cp.zeros((4, 2)),
                'H': cp.eye(4),
                'Q': cp.eye(4) * 0.1,
                'R': cp.eye(4) * 0.1,
                'x0': cp.zeros((4, 1)),
                'P0': cp.eye(4)
            }

    def update(self, detected_bboxes):
        for tracker in self.trackers:
            tracker.predict()

        trks = cp.zeros((len(self.trackers), 5))
        to_del = []
        for t, tracker in enumerate(trks):
            pos = self.trackers[t].get_state().flatten()
            trks[t, :] = [pos[0], pos[1], pos[2], pos[3], 0]
            if cp.any(cp.isnan(pos)):
                to_del.append(t)
        trks = cp.compress_rows(cp.masked_invalid(trks))
        for t in reversed(to_del):
            self.trackers.pop(t)

        matched, unmatched_detections, unmatched_trackers = associate_detections_to_trackers_gpu(detections, trks, self.iou_threshold)

        for m in matched:
            self.trackers[m[1]].update(detections[m[0], :])

        for i in unmatched_detections:
            kf = self.kalman_filter_class(**self.kalman_filter_params)
            kf.update(detections[i, :])
            self.trackers.append(kf)

        for trk in reversed(self.trackers):
            if (trk.last_update_time > self.max_age):
                self.trackers.pop(len(self.trackers) - 1)

        ret = []
        for trk in self.trackers:
            if trk.hits >= self.min_hits or self.frame_count <= self.min_hits:
                ret.append(cp.concatenate((trk.get_state()[0], [trk.id + 1])).reshape(1, -1))
        if len(ret) > 0:
            return cp.concatenate(ret)
        return cp.empty((0, 5))
